<a href="https://colab.research.google.com/github/kimhyeongmin-khu/MLMovieProj/blob/main/%EC%98%81%ED%99%94%EC%A7%84%ED%9D%A5%EC%9C%84%EC%9B%90%ED%9A%8C_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
import os
import sys
import urllib.request
import json
import pandas as pd
from datetime import datetime, timedelta

client_id = "zpxnQpa1UTcH7rTjlSJy"
client_secret = "UvLNC2Ow_i"
url = "https://openapi.naver.com/v1/datalab/search"

df = pd.read_csv('output_to_trend.csv')
final_df_list = []

for index, row in df.iterrows():
    movie_title = row['Movie_Title']
    release_date = datetime.strptime(row['Release_Date'], '%Y-%m-%d')

    # 개봉일 전후 1주일 기간 설정
    end_date = (release_date + timedelta(days=6)).strftime('%Y-%m-%d')
    start_date = release_date.strftime('%Y-%m-%d')

    # JSON 형식의 본문 문자열을 수정
    body = f"""
    {{
        "startDate": "{start_date}",
        "endDate": "{end_date}",
        "timeUnit": "date",
        "keywordGroups": [
            {{
                "groupName": "{movie_title}",
                "keywords": ["{movie_title}"]
            }}
        ]
    }}
    """

    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id", client_id)
    request.add_header("X-Naver-Client-Secret", client_secret)
    request.add_header("Content-Type", "application/json")

    try:
        response = urllib.request.urlopen(request, data=body.encode("utf-8"))
        rescode = response.getcode()
        if rescode == 200:
            response_body = response.read()
            response_data = json.loads(response_body.decode('utf-8'))

            # 결과 리스트에서 데이터 추출
            results = response_data['results'][0]
            title = results['title']
            data = results['data']

            # 데이터프레임 생성
            df_temp = pd.DataFrame(data)

            # period를 날짜 순서로 정렬하고, 인덱스 재설정
            df_temp = df_temp.sort_values(by='period').reset_index(drop=True)

            # 인덱스를 1부터 7로 설정
            df_temp.index = range(1, len(df_temp) + 1)

            # 컬럼 이름 설정
            df_temp.columns = ['column', title]

            # 컬럼을 1부터 7로 설정
            df_temp['column'] = range(1, len(df_temp) + 1)

            # 최종 데이터프레임에 추가
            final_df_list.append(df_temp.set_index('column').T)

        else:
            print("Error Code:" + str(rescode))
    except Exception as e:
        print(f"Error: {e}")

# 모든 데이터프레임을 수직으로 결합
final_df = pd.concat(final_df_list)

# 결과를 CSV 파일로 저장
final_df.to_csv('combined_trend_data.csv', index=True)

# 결과 출력
print(final_df)


In [ ]:
#중요코드
import requests
import json
import pandas as pd

# CSV 파일 경로 설정
path = '0530_dataset/preprosessing_ver5.csv'

# CSV 파일 읽기
df = pd.read_csv(path)

# movieCd를 저장할 새로운 열 추가
df['first_act'] = None
df['second_act'] = None

# API 키
api_key = "0ce87dc872a5a617a94149263d2bb648"

for index, row in df.iterrows():
    mvcd=row['movieCd']

    # 요청 URL 및 파라미터
    url = "http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieInfo.json"
    params = {
        'key': api_key,
        'movieCd': mvcd
    }

    # GET 요청
    response = requests.get(url, params=params)

    # 응답 확인
    if response.status_code == 200:
        data = response.json()

        # actors 리스트에서 첫 번째와 두 번째 peopleNm 가져오기
        actors = data['movieInfoResult']['movieInfo']['actors']

        if len(actors) >= 2:
            first_actor = actors[0]['peopleNm']
            second_actor = actors[1]['peopleNm']
            df.at[index,'first_act'] = first_actor
            df.at[index,'second_act'] = second_actor

            print(f"First Actor: {first_actor}")
            print(f"Second Actor: {second_actor}")
        else:
            print("Less than 2 actors available in the data.")
            first_actor = actors[0]['peopleNm']
            df.at[index,'first_act'] = first_actor
    else:
        print(f"Error: {response.status_code}")

df.to_csv("preprosessing_ver6.csv", index=False)

In [ ]:
#중요코드
import requests
import pandas as pd

# CSV 파일 경로 설정
path = '0530_dataset/preprosessing_ver5.csv'

# CSV 파일 읽기
df = pd.read_csv(path)
df['Director_code'] = None

# API 키
api_key = "f73414c46c8384565dcdb519d1156297"

for index, row in df.iterrows():
    director = row['Director'].split(',')[0]
    mvnm = row['Movie_Title']  # 영화 제목에서 첫 번째 항목 선택

    # 요청 URL 및 파라미터
    url = "http://www.kobis.or.kr/kobisopenapi/webservice/rest/people/searchPeopleList.json"
    params = {
        'key': api_key,
        'peopleNm': director,
        'itemPerPage' : 100
    }

    # GET 요청
    response = requests.get(url, params=params)

    # 응답 확인
    if response.status_code == 200:
        data = response.json()

        # peopleList에서 감독 정보를 찾기
        for person in data['peopleListResult']['peopleList']:
            if 'filmoNames' in person:
                if mvnm in person['filmoNames']:
                    df.at[index, 'Director_code'] = person['peopleCd']
                    if person['repRoleNm'] != '감독':
                        print(person)
                    break  # 첫 번째 일치하는 결과만 저장
    else:
        print(f"Failed to fetch data for {director}. Status code: {response.status_code}")

# 데이터프레임 저장 또는 확인
df.to_csv('0530_dataset/preprosessing_ver5_r2.csv', index=False)
print(df.head())


In [ ]:
#중요코드
import requests
import pandas as pd

# CSV 파일 경로 설정
path = '0530_dataset/preprosessing_ver8.csv'

# CSV 파일 읽기
df = pd.read_csv(path)
df['di_ca'] = None

# API 키
api_key = "f73414c46c8384565dcdb519d1156297"

for index, row in df.iterrows():
    director = row['Director_code']

    # 요청 URL 및 파라미터
    url = "http://www.kobis.or.kr/kobisopenapi/webservice/rest/people/searchPeopleInfo.json"
    params = {
        'key': api_key,
        'peopleCd': director
    }

    # GET 요청
    response = requests.get(url, params=params)

    # 응답 확인
    if response.status_code == 200:
        data = response.json()

        director_codes = []
        person = data['peopleInfoResult']['peopleInfo']
        if 'filmos' in person:
            for film in person['filmos']:
                if film['moviePartNm'] == '감독':
                    director_codes.append(film['movieNm'])

        if director_codes:
            df.at[index, 'di_ca'] = ','.join(director_codes)

# 데이터프레임 저장 또는 확인
df.to_csv('0530_dataset/preprosessing_ver8_r1.csv', index=False)
print(df.head())


In [ ]:
#중요코드
import requests
import pandas as pd

# CSV 파일 경로 설정
path = '0530_dataset/preprosessing_ver5_r3.csv'

# CSV 파일 읽기
df = pd.read_csv(path)
df['Actor_code_1'] = None

# API 키
api_key = "f73414c46c8384565dcdb519d1156297"

for index, row in df.iterrows():
    director = row['Director'].split(',')[0]
    mvnm = row['Movie_Title']  # 영화 제목에서 첫 번째 항목 선택

    # 요청 URL 및 파라미터
    url = "http://www.kobis.or.kr/kobisopenapi/webservice/rest/people/searchPeopleList.json"
    params = {
        'key': api_key,
        'peopleNm': director,
        'itemPerPage' : 100
    }

    # GET 요청
    response = requests.get(url, params=params)

    # 응답 확인
    if response.status_code == 200:
        data = response.json()

        # peopleList에서 감독 정보를 찾기
        for person in data['peopleListResult']['peopleList']:
            if 'filmoNames' in person:
                if mvnm in person['filmoNames']:
                    df.at[index, 'Director_code'] = person['peopleCd']
                    if person['repRoleNm'] != '감독':
                        print(person)
                    break  # 첫 번째 일치하는 결과만 저장
    else:
        print(f"Failed to fetch data for {director}. Status code: {response.status_code}")

# 데이터프레임 저장 또는 확인
df.to_csv('0530_dataset/preprosessing_ver5_r2.csv', index=False)
print(df.head())


In [ ]:
#중요코드
import requests
import pandas as pd

# CSV 파일 경로 설정
path = '0530_dataset/preprosessing_ver7.csv'

# CSV 파일 읽기
df = pd.read_csv(path)
df['first_act_code'] = None

# API 키
api_key = "f73414c46c8384565dcdb519d1156297"

for index, row in df.iterrows():
    actor = row['first_act']
    mvnm = row['Movie_Title']  # 영화 제목에서 첫 번째 항목 선택

    # 요청 URL 및 파라미터
    url = "http://www.kobis.or.kr/kobisopenapi/webservice/rest/people/searchPeopleList.json"
    params = {
        'key': api_key,
        'peopleNm': actor,
        'itemPerPage' : 100
    }

    # GET 요청
    response = requests.get(url, params=params)

    # 응답 확인
    if response.status_code == 200:
        data = response.json()

        # peopleList에서 감독 정보를 찾기
        for person in data['peopleListResult']['peopleList']:
            if 'filmoNames' in person:
                if mvnm in person['filmoNames']:
                    df.at[index, 'first_act_code'] = person['peopleCd']
                    if person['repRoleNm'] != '배우':
                        print(person)
                    break  # 첫 번째 일치하는 결과만 저장
    else:
        print(f"Failed to fetch data for {actor}. Status code: {response.status_code}")

# 데이터프레임 저장 또는 확인
df.to_csv('0530_dataset/preprosessing_ver7_r2.csv', index=False)
print(df.head())


In [ ]:
#중요코드
import requests
import pandas as pd

# CSV 파일 경로 설정
path = '0530_dataset/preprosessing_ver7_r2.csv'

# CSV 파일 읽기
df = pd.read_csv(path)
df['second_act_code'] = None

# API 키
api_key = "f73414c46c8384565dcdb519d1156297"

for index, row in df.iterrows():
    actor = row['second_act']
    mvnm = row['Movie_Title']  # 영화 제목에서 첫 번째 항목 선택

    # 요청 URL 및 파라미터
    url = "http://www.kobis.or.kr/kobisopenapi/webservice/rest/people/searchPeopleList.json"
    params = {
        'key': api_key,
        'peopleNm': actor,
        'itemPerPage' : 100
    }

    # GET 요청
    response = requests.get(url, params=params)

    # 응답 확인
    if response.status_code == 200:
        data = response.json()

        # peopleList에서 감독 정보를 찾기
        for person in data['peopleListResult']['peopleList']:
            if 'filmoNames' in person:
                if mvnm in person['filmoNames']:
                    df.at[index, 'second_act_code'] = person['peopleCd']
                    if person['repRoleNm'] != '배우':
                        print(person)
                    break  # 첫 번째 일치하는 결과만 저장
    else:
        print(f"Failed to fetch data for {actor}. Status code: {response.status_code}")

# 데이터프레임 저장 또는 확인
df.to_csv('0530_dataset/preprosessing_ver7_r3.csv', index=False)
print(df.head())


In [ ]:
#중요코드
import requests
import pandas as pd

# CSV 파일 경로 설정
path = '0530_dataset/preprosessing_ver7_r3.csv'

# CSV 파일 읽기
df = pd.read_csv(path)
df['Pc_code'] = None

# API 키
api_key = "f73414c46c8384565dcdb519d1156297"

for index, row in df.iterrows():
    maker = row['Production_Company']
    mvnm = row['Movie_Title']  # 영화 제목에서 첫 번째 항목 선택

    # 요청 URL 및 파라미터
    url = "http://www.kobis.or.kr/kobisopenapi/webservice/rest/company/searchCompanyList.json"
    params = {
        'key': api_key,
        'companyNm': maker,
        'itemPerPage' : 100
    }

    # GET 요청
    response = requests.get(url, params=params)

    # 응답 확인
    if response.status_code == 200:
        data = response.json()

        # companyList에서 영화사 정보를 찾기
        for company in data['companyListResult']['companyList']:
            if 'filmoNames' in company:
                if mvnm in company['filmoNames']:
                    df.at[index, 'Pc_code'] = company['companyCd']
                    break  # 첫 번째 일치하는 결과만 저장
    else:
        print(f"Failed to fetch data for {maker}. Status code: {response.status_code}")

# 데이터프레임 저장 또는 확인
df.to_csv('0530_dataset/preprosessing_ver7_r4.csv', index=False)
print(df.head())


In [ ]:
#중요코드
import requests
import pandas as pd

# CSV 파일 경로 설정
path = '0530_dataset/preprosessing_ver7_r6.csv'

# CSV 파일 읽기
df = pd.read_csv(path)
df['Distributor_code'] = None

# API 키
api_key = "f73414c46c8384565dcdb519d1156297"

for index, row in df.iterrows():
    i=0
    maker = row['one_Distributor']
    mvnm = row['Movie_Title']  # 영화 제목에서 첫 번째 항목 선택
    if maker =="메가박스중앙(주)":
        maker="플러스엠 엔터테인먼트"
        i+=1


    # 요청 URL 및 파라미터
    url = "http://www.kobis.or.kr/kobisopenapi/webservice/rest/company/searchCompanyList.json"
    params = {
        'key': api_key,
        'companyNm': maker,
        'itemPerPage' : 100
    }

    # GET 요청
    response = requests.get(url, params=params)

    # 응답 확인
    if response.status_code == 200:
        data = response.json()

        # companyList에서 영화사 정보를 찾기
        for company in data['companyListResult']['companyList']:
            if 'filmoNames' in company:
                if mvnm in company['filmoNames'] or i==1:
                    df.at[index, 'Distributor_code'] = company['companyCd']
                    break  # 첫 번째 일치하는 결과만 저장
            print(f"배급사 {maker} 없음")
    else:
        print(f"Failed to fetch data for {maker}. Status code: {response.status_code}")

# 데이터프레임 저장 또는 확인
df.to_csv('0530_dataset/preprosessing_ver7_r7.csv', index=False)
print(df.head())


In [ ]:
#중요코드
import requests
import pandas as pd

# CSV 파일 경로 설정
path = '0530_dataset/preprosessing_ver8_r1.csv'

# CSV 파일 읽기
df = pd.read_csv(path)
df['Distributors_mv'] = None

# API 키
api_key = "f73414c46c8384565dcdb519d1156297"

for index, row in df.iterrows():
    distributor = row['Distributor_code']

    # 요청 URL 및 파라미터
    url = "http://www.kobis.or.kr/kobisopenapi/webservice/rest/company/searchCompanyInfo.json"
    params = {
        'key': api_key,
        'companyCd': distributor
    }

    # GET 요청
    response = requests.get(url, params=params)

    # 응답 확인
    if response.status_code == 200:
        data = response.json()

        company_codes = []
        company = data['companyInfoResult']['companyInfo']
        if 'filmos' in company:
            for film in company['filmos']:
                if film['companyPartNm'] == '배급사':
                    company_codes.append(film['movieNm'])

        if company_codes:
            df.at[index, 'Distributors_mv'] = ','.join(company_codes)

# 데이터프레임 저장 또는 확인
df.to_csv('0530_dataset/preprosessing_ver8_r2.csv', index=False)
print(df.head())


In [ ]:
#중요코드
import requests
import pandas as pd

# CSV 파일 경로 설정
path = '0530_dataset/preprosessing_ver8_r2.csv'

# CSV 파일 읽기
df = pd.read_csv(path)
df['first_actor_mv'] = None

# API 키
api_key = "f73414c46c8384565dcdb519d1156297"

for index, row in df.iterrows():
    actor = row['first_act_code']

    # 요청 URL 및 파라미터
    url = "http://www.kobis.or.kr/kobisopenapi/webservice/rest/people/searchPeopleInfo.json"
    params = {
        'key': api_key,
        'peopleCd': actor
    }

    # GET 요청
    response = requests.get(url, params=params)

    # 응답 확인
    if response.status_code == 200:
        data = response.json()

        actor_codes = []
        person = data['peopleInfoResult']['peopleInfo']
        if 'filmos' in person:
            for film in person['filmos']:
                if film['moviePartNm'] == '배우':
                    actor_codes.append(film['movieNm'])

        if actor_codes:
            df.at[index, 'first_actor_mv'] = ','.join(actor_codes)

# 데이터프레임 저장 또는 확인
df.to_csv('0530_dataset/preprosessing_ver8_r3.csv', index=False)
print(df.head())


In [ ]:
#중요코드
import requests
import pandas as pd

# CSV 파일 경로 설정
path = '0530_dataset/preprosessing_ver8_r3.csv'

# CSV 파일 읽기
df = pd.read_csv(path)
df['second_actor_mv'] = None

# API 키
api_key = "0ce87dc872a5a617a94149263d2bb648"

for index, row in df.iterrows():
    actor = row['second_act_code']
    if pd.isna(actor) or actor == '':
        continue
    # 요청 URL 및 파라미터
    url = "http://www.kobis.or.kr/kobisopenapi/webservice/rest/people/searchPeopleInfo.json"
    params = {
        'key': api_key,
        'peopleCd': int(actor)
    }
    print(actor)
    # GET 요청
    response = requests.get(url, params=params)

    # 응답 확인
    if response.status_code == 200:
        data = response.json()

        actor_codes = []
        person = data['peopleInfoResult']['peopleInfo']
        if 'filmos' in person:
            for film in person['filmos']:
                if film['moviePartNm'] == '배우':
                    actor_codes.append(film['movieNm'])

        if actor_codes:
            df.at[index, 'second_actor_mv'] = ','.join(actor_codes)

# 데이터프레임 저장 또는 확인
df.to_csv('0530_dataset/preprosessing_ver8_r4.csv', index=False)
print(df.head())
